In [39]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [40]:
columns = ["COD_HOSPITAL"
,"CIP_ENCRIPTADO"
,"SEXO"
,"FECHA_NACIMIENTO"
,"PROVINCIA"
,"COMUNA"
,"NACIONALIDAD"
,"PREVISION"
,"SERVICIO_SALUD"
,"TIPO_PROCEDENCIA"
,"TIPO_INGRESO"
,"ESPECIALIDAD_MEDICA"
,"TIPO_ACTIVIDAD"
,"FECHA_INGRESO"
,"SERVICIOINGRESO"
,"FECHAALTA"
,"SERVICIOALTA"
,"TIPOALTA"
,"DIAGNOSTICO1"
,"DIAGNOSTICO2"
,"DIAGNOSTICO3"
,"DIAGNOSTICO4"
,"DIAGNOSTICO5"
,"DIAGNOSTICO6"
,"DIAGNOSTICO7"
,"DIAGNOSTICO8"
,"DIAGNOSTICO9"
,"DIAGNOSTICO10"
,"PROCEDIMIENTO1"
,"PROCEDIMIENTO2"
,"PROCEDIMIENTO3"
,"PROCEDIMIENTO4"
,"PROCEDIMIENTO5"
,"PROCEDIMIENTO6"
,"PROCEDIMIENTO7"
,"PROCEDIMIENTO8"
,"PROCEDIMIENTO9"
,"PROCEDIMIENTO10"
,"USOSPABELLON"
,"IR_29301_COD_GRD"
,"IR_29301_PESO"]

In [41]:
data = pd.read_csv(r'Datos\GRD_PUBLICO_2023.csv',sep='|',engine='python',encoding='utf-16', usecols=columns)

In [48]:
def autoconvert_datetime(value):
    formats = ['%m/%d/%Y', '%m-%d-%y', '%m-%d-%Y', '%Y-%m-%d', '%Y/%m/%d', '%Y%m%d', '%d/%m/%Y', '%d-%m-%y', '%d-%m-%Y']
    result_format = '%Y-%m-%d'
    for dt_format in formats:
        try:
            dt_obj = pd.datetime.strptime(value, dt_format)
            return dt_obj.strftime(result_format)
        except Exception as e:
            pass
    return value

In [49]:
data[['FECHAALTA','FECHA_INGRESO','FECHA_NACIMIENTO']] = data[['FECHAALTA','FECHA_INGRESO','FECHA_NACIMIENTO']].apply(autoconvert_datetime)


In [50]:
data

,COD_HOSPITAL,CIP_ENCRIPTADO,SEXO,FECHA_NACIMIENTO,PROVINCIA,COMUNA,NACIONALIDAD,PREVISION,SERVICIO_SALUD,TIPO_PROCEDENCIA,...,PROCEDIMIENTO4,PROCEDIMIENTO5,PROCEDIMIENTO6,PROCEDIMIENTO7,PROCEDIMIENTO8,PROCEDIMIENTO9,PROCEDIMIENTO10,USOSPABELLON,IR_29301_COD_GRD,IR_29301_PESO
0,122100,72196813,HOMBRE,1980-04-13,VALDIVIA,PANGUIPULLI,CHILE,FONASA INSTITUCIONAL - (MAI) A,VALDIVIA,SERVICIO EMERGENCIA (DOMICILIO),...,96.59,90.59,90.39,99.29,99.21,99.18,47.01,NaN,064161,"0,7094"
1,114101,DESCONOCIDO,HOMBRE,2001-11-04,CORDILLERA,PUENTE ALTO,VENEZUELA (REPÚBLICA BOLIVARIANA DE),PARTICULAR,METROPOLITANO SURORIENTE,SERVICIO EMERGENCIA (DOMICILIO),...,87.44,93.96,93.18,99.19,99.21,99.26,99.29,1.0,041301,"1,0869"
2,114101,97627974,HOMBRE,2020-07-30,CORDILLERA,PUENTE ALTO,CHILE,FONASA INSTITUCIONAL - (MAI) A,METROPOLITANO SURORIENTE,"APS URGENCIA (SAPU, SUR, SUC)",...,90.59,90.39,93.18,NaN,NaN,NaN,NaN,NaN,044161,"0,4084"
3,106103,67770674,MUJER,1978-11-14,SAN ANTONIO,SAN ANTONIO,CHILE,FONASA INSTITUCIONAL - (MAI) C,VALPARAISO SAN ANTONIO,PLAN DE RESOLUCIÓN LE,...,57.94,89.7,NaN,NaN,NaN,NaN,NaN,1.0,131301,"0,6406"
4,106103,100390800,MUJER,2023-12-20,SAN ANTONIO,SAN ANTONIO,CHILE,FONASA INSTITUCIONAL - (MAI) B,VALPARAISO SAN ANTONIO,SERVICIO EMERGENCIA (DOMICILIO),...,89.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,074131,"0,5346"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039582,105102,78111817,MUJER,1995-12-06,LIMARI,OVALLE,CHILE,FONASA INSTITUCIONAL - (MAI) D,COQUIMBO,SERVICIO EMERGENCIA (DOMICILIO),...,90.59,99.04,89.08,99.29,39.96,99.21,93.59,5.0,131111,"0,5227"
1039583,118100,76593606,MUJER,1991-04-26,CONCEPCION,CONCEPCIÓN,CHILE,FONASA INSTITUCIONAL - (MAI) D,CONCEPCIÓN,"CENTRO ESPECIALIDADES (CDT, CRS, CONSULTORIO A...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,132340,"0,1761"
1039584,115100,100360940,HOMBRE,2015-01-08,CACHAPOAL,RANCAGUA,CHILE,FONASA INSTITUCIONAL - (MAI) A,LIBERTADOR B. O HIGGINS,SERVICIO EMERGENCIA (DOMICILIO),...,88.19,99.29,99.21,99.18,NaN,NaN,NaN,1.0,061131,"0,7398"
1039585,123100,78557002,HOMBRE,2014-01-07,OSORNO,OSORNO,CHILE,FONASA INSTITUCIONAL - (MAI) A,OSORNO,"CENTRO ESPECIALIDADES (CDT, CRS, CONSULTORIO A...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,044181,"0,3452"
